In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
# import logging
# import warnings

# logging.basicConfig(level=logging.DEBUG)
# warnings.filterwarnings('ignore')

In [4]:
# import tensorflow as tf

# tf.compat.v1.enable_eager_execution()

In [5]:
import malaya_speech
import malaya_speech.config
from malaya_speech.train.model import glowtts
import tensorflow as tf
import numpy as np
import math






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [6]:
config = malaya_speech.config.fastspeech2_config
config['encoder_hidden_size'] = 192
config['encoder_num_hidden_layers'] = 6
config['encoder_attention_head_size'] = 32
config['encoder_intermediate_size'] = 768
config = glowtts.Config(vocab_size = 66, **config)
config_glowtts = glowtts.Config_GlowTTS(malaya_speech.config.glowtts_config)

In [7]:
i = tf.placeholder(tf.int32, [None, None])
i_lengths = tf.placeholder(tf.int32, [None])
mel_outputs = tf.placeholder(tf.float32, [None, None, 80])
mel_lengths = tf.placeholder(tf.int32, [None])

In [8]:
import pickle

with open('dataset-mel.pkl', 'rb') as fopen:
    data, d = pickle.load(fopen)
    
with open('dataset-mel-wav.pkl', 'rb') as fopen:
    wav = pickle.load(fopen)
    
data.keys()

dict_keys(['mel', 'text_ids', 'len_mel', 'len_text_ids', 'stop_token_target', 'f0', 'len_f0', 'energy', 'len_energy', 'g'])

In [9]:
# i = tf.convert_to_tensor(data['text_ids'])
# mel_outputs = tf.convert_to_tensor(data['mel'].astype(np.float32))
# mel_lengths = tf.convert_to_tensor(data['len_mel'][:,0])

In [10]:
model = glowtts.Model(config, config_glowtts)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor




In [11]:
(z, z_m, z_logs, logdet, z_mask), (x_m, x_logs, x_mask), (attn, logw, logw_) = model(i, 
                                                                                     gen = True, training = True)
(z, z_m, z_logs, logdet, z_mask), (x_m, x_logs, x_mask), (attn, logw, logw_)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



((<tf.Tensor 'glowtts/FlowSpecDecoder/mul_4:0' shape=(?, ?, 80) dtype=float32>,
  <tf.Tensor 'glowtts/MatMul:0' shape=(?, ?, 80) dtype=float32>,
  <tf.Tensor 'glowtts/MatMul_1:0' shape=(?, ?, 80) dtype=float32>,
  None,
  <tf.Tensor 'glowtts/transpose:0' shape=(?, ?, 1) dtype=float32>),
 (<tf.Tensor 'glowtts/Encoder/mul:0' shape=(?, ?, 80) dtype=float32>,
  <tf.Tensor 'glowtts/Encoder/zeros_like:0' shape=(?, ?, 80) dtype=float32>,
  <tf.Tensor 'glowtts/Encoder/ExpandDims:0' shape=(?, ?, 1) dtype=float32>),
 (<tf.Tensor 'glowtts/ExpandDims_2:0' shape=(?, 1, ?, ?) dtype=float32>,
  <tf.Tensor 'glowtts/Encoder/ExpandDims_1:0' shape=(?, ?, 1) dtype=float32>,
  <tf.Tensor 'glowtts/mul_6:0' shape=(?, ?, ?) dtype=float32>))

In [12]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [13]:
len(tf.trainable_variables())

504

In [14]:
import collections
import re

def get_assignment_map_from_checkpoint(tvars, init_checkpoint, logging=True):
    """Compute the union of the current variables and checkpoint variables."""
    assignment_map = {}
    initialized_variable_names = {}

    name_to_variable = collections.OrderedDict()
    for var in tvars:
        name = var.name
        m = re.match('^(.*):\\d+$', name)
        if m is not None:
            name = m.group(1)
        name_to_variable[name] = var

    init_vars = tf.train.list_variables(init_checkpoint)

    assignment_map = collections.OrderedDict()
    for x in init_vars:
        (name, var) = (x[0], x[1])
        if name not in name_to_variable:
            continue

        assignment_map[name] = name
        assignment_map[name] = name_to_variable[name]
        initialized_variable_names[name] = 1
        initialized_variable_names[name + ':0'] = 1

    if logging:
        tf.logging.info('**** Trainable Variables ****')
        for var in tvars:
            init_string = ''
            if var.name in initialized_variable_names:
                init_string = ', *INIT_FROM_CKPT*'
            tf.logging.info(
                '  name = %s, shape = %s%s', var.name, var.shape, init_string
            )

    return (assignment_map, initialized_variable_names)

In [15]:
assignment_map, _ = get_assignment_map_from_checkpoint(tf.trainable_variables(), 'test/model.ckpt')
len(assignment_map)

INFO:tensorflow:**** Trainable Variables ****


INFO:tensorflow:**** Trainable Variables ****


INFO:tensorflow:  name = n_blocks_1/logs:0, shape = (1, 1, 160), *INIT_FROM_CKPT*


INFO:tensorflow:  name = n_blocks_1/logs:0, shape = (1, 1, 160), *INIT_FROM_CKPT*


INFO:tensorflow:  name = n_blocks_1/bias:0, shape = (1, 1, 160), *INIT_FROM_CKPT*


...

INFO:tensorflow:  name = glowtts/FlowSpecDecoder/CouplingBlock_0/conv1d_3/kernel:0, shape = (1, 192, 160), *INIT_FROM_CKPT*


INFO:tensorflow:  name = glowtts/FlowSpecDecoder/CouplingBlock_0/conv1d_3/kernel:0, shape = (1, 192, 160), *INIT_FROM_CKPT*


INFO:tensorflow:  name = glowtts/FlowSpecDecoder/CouplingBlock_0/conv1d_3/bias:0, shape = (160,), *INIT_FROM_CKPT*


INFO:tensorflow:  name = glowtts/FlowSpecDecoder/CouplingBlock_0/conv1d_3/bias:0, shape = (160,), *INIT_FROM_CKPT*


504

In [35]:
saver = tf.train.Saver(assignment_map)
saver.restore(sess, 'test/model.ckpt')

INFO:tensorflow:Restoring parameters from test/model.ckpt


INFO:tensorflow:Restoring parameters from test/model.ckpt


In [36]:
o_ = sess.run(z, feed_dict = {i: data['text_ids']})
o_.shape

(1, 148, 80)

In [37]:
o_ = sess.run(z_mask, feed_dict = {i: data['text_ids']})
o_.shape

(1, 156, 1)

In [38]:
o_ = sess.run(logw, feed_dict = {i: data['text_ids']})
o_.shape

(1, 72, 1)

In [39]:
np.sum(np.sum(np.ceil(np.exp(o_)), axis = 2), axis = 1)

array([166.], dtype=float32)

In [40]:
!ls -lh test
!rm -rf test

total 200776
-rw-r--r--  1 huseinzolkepli  staff    77B Aug 22 22:50 checkpoint
-rw-r--r--  1 huseinzolkepli  staff    81M Aug 22 22:50 model.ckpt.data-00000-of-00001
-rw-r--r--  1 huseinzolkepli  staff    20K Aug 22 22:50 model.ckpt.index
-rw-r--r--  1 huseinzolkepli  staff    17M Aug 22 22:50 model.ckpt.meta
